# Simulated Instrument Signal

In this lesson we cover three TOAST operators for simulating noise and systematics:
* `OpSimNoise` for simulating instrumental noise
* `OpSimAtmosphere` for simulating atmospheric noise
* `OpSimSSS` for simulating scan-synchronous signal (typically ground pick-up)

In [ ]:
# Are you using a special reservation for a workshop?
# If so, set it here:
nersc_reservation = None

# Load common tools for all lessons
import sys
sys.path.insert(0, "..")
from lesson_tools import (
    check_nersc,
    fake_focalplane
)
nersc_host, nersc_repo = check_nersc(reservation=nersc_reservation)
if nersc_host is not None:
    %reload_ext slurm_magic

We begin by generating an observing schedule containing a single observation

In [ ]:
! toast_ground_schedule.py \
    --site-lat "-22.958064" \
    --site-lon "-67.786222" \
    --site-alt 5200 \
    --site-name Atacama \
    --telescope LAT \
    --start "2020-01-01 00:00:00" \
    --stop "2020-01-01 02:00:00" \
    --patch-coord C \
    --patch small_patch,1,40,-40,44,-44 \
    --ces-max-time 86400 \
    --out schedule.txt

! cat schedule.txt

In [ ]:
! [[ ! -e weather_Atacama.fits ]] && wget http://portal.nersc.gov/project/cmb/toast_data/example_data/weather_Atacama.fits

Typical TOAST pipelines like `toast_ground_sim.py` create synthetic observations by loading the schedule from file and then creating instances of `TODGround` according to the observing schedule. This is most easily accomplished with `toast.pipeline_tools`.

In [ ]:
import toast
from toast.mpi import MPI

import numpy as np
import matplotlib.pyplot as plt

from toast import pipeline_tools 

mpiworld, procs, rank = toast.mpi.get_world()
comm = toast.mpi.Comm(mpiworld)

# Create a fake focalplane, we could also load one from file

focalplane = toast.pipeline_tools.Focalplane(fake_focalplane())

# A pipeline would create the args object with argparse

class args:
    split_schedule = None
    schedule = "schedule.txt"
    sort_schedule = False  # Matters for parallelization
    weather = "weather_Atacama.fits"
    sample_rate = 10  # Hz
    scan_rate = 1.0  # deg / s
    scan_accell = 10.0  # deg / s^2
    hwp_rpm = None
    hwp_step_deg = None
    hwp_step_time_s = None
    
# Load the observing schedule, append weather and focalplane to it
    
schedules = toast.pipeline_tools.load_schedule(args, comm)
toast.pipeline_tools.load_weather(args, comm, schedules)
# There could be more than one observing schedule, but not this time
schedule = schedules[0]
schedule.telescope.focalplane = focalplane

# Create a TODGround object and embed it in an observation

ces = schedule.ceslist[0]  # We know there is only one CES
totsamples = int((ces.stop_time - ces.start_time) * args.sample_rate)

# create the TOD for this observation

if comm.comm_group is not None:
    # Available detectors should be split between processes in the group
    ndetrank = comm.comm_group.size
else:
    ndetrank = 1

tod = toast.todmap.TODGround(
    comm.comm_group,
    telescope.focalplane.detquats,
    totsamples,
    detranks=ndetrank,
    firsttime=ces.start_time,
    rate=args.sample_rate,
    site_lon=telescope.site.lon,
    site_lat=telescope.site.lat,
    site_alt=telescope.site.alt,
    azmin=ces.azmin,
    azmax=ces.azmax,
    el=ces.el,
    scanrate=args.scan_rate,
    scan_accel=args.scan_accel,
    #CES_start=None,
    #CES_stop=None,
    #sun_angle_min=args.sun_angle_min,
    #coord=args.coord,
    #sampsizes=None,
    #report_timing=args.debug,
    hwprpm=args.hwp_rpm,
    hwpstep=args.hwp_step_deg,
    hwpsteptime=args.hwp_step_time_s,
)
    

## Running in Parallel

The NERSC login nodes do not support MPI, so all of the previous examples are running serially.  To run in parallel, we can submit a batch job version of the above examples:

In [ ]:
%%writefile siminst.py

import toast
from toast.mpi import MPI


In [ ]:
if nersc_host is not None:
    %srun -N 1 -C knl -n 32 -c 2 --cpu_bind=cores -t 00:03:00 python siminst.py